# VGG for MNIST classification

In [1]:
import sys
import tensorflow as tf
import numpy as np
from matplotlib.pyplot import matplotlib

sys.path.append("..")

from dataset.dataset.opensets import MNIST
from vgg import VGGModel

%matplotlib inline

In [2]:
BATCH_SIZE = 256
mnist = MNIST()

Extracting ExtractingExtracting/tmp/train-images-idx3-ubyte.gz Extracting
 /tmp/train-labels-idx1-ubyte.gz /tmp/t10k-labels-idx1-ubyte.gz/tmp/t10k-images-idx3-ubyte.gz




In [3]:
model_config = {'session': {'config': tf.ConfigProto()},
                'loss': 'sigmoid_cross_entropy',
                'optimizer': {'name':'Adam',
                              'use_locking': True},
                'vgg_arch': 'VGG19',
                'images_shape': (28, 28, 1),
                'n_classes': 10,
                'b_norm': True,
                'momentum': 0.1}

feed_dict_train={'images': 'images', 
                 'labels': 'labels',
                 'training': True}

feed_dict_test={'images': 'images', 
                'labels': 'labels',
                'training': False}

In [4]:
train_pp = (mnist.train.p
            .init_variable('loss_history', init_on_each_run=list)
            .init_variable('current_loss', init_on_each_run=0)
            .init_variable('pred_label', init_on_each_run=list)
            .init_model('static', VGGModel, 'VGG', config=model_config)
            .train_model('VGG', 
                         fetches=['loss', 'predicted_labels'], 
                         feed_dict=feed_dict_train, 
                         save_to=['current_loss', 'pred_label'])
            .save_to_variable('loss_history', var='current_loss', mode='a'))

train_pp.run(BATCH_SIZE, shuffle=True, n_epochs=1, drop_last=False)

INFO:tensorflow:logits.dtype=<dtype: 'float32'>.
INFO:tensorflow:multi_class_labels.dtype=<dtype: 'float32'>.
INFO:tensorflow:losses.dtype=<dtype: 'float32'>.


In [7]:
test_pp = (mnist.test.p
           .import_model('VGG', train_pp)
           .init_variable('all_predictions', init_on_each_run=list)
           .predict_model('VGG',
                          fetches='predicted_labels',
                          feed_dict=feed_dict_test,
                          append_to='all_predictions'))

batch = test_pp.next_batch(10, shuffle=True, n_epochs=1, drop_last=False)

In [8]:
print('Predictions: ', test_pp.get_variable('all_predictions')[0])
print('Labels: ', batch.data.labels)

Predictions:  [7 7 0 0 1 7 7 1 0 0]
Labels:  [7 7 6 0 1 4 7 1 6 6]
